In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
np.set_printoptions(suppress=True)
df = pd.read_csv('data/covid_19_data.csv')

In [2]:
# I referred code regarding data preprocessing from https://www.kaggle.com/chirag9073/coronavirus-covid-19-outbreak-data-analysis

df.drop(['SNo'], axis=1, inplace=True)
df['ObservationDate'] = df['ObservationDate'].apply(pd.to_datetime)

In [3]:
df.head(10)

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
5,2020-01-22,Guangdong,Mainland China,1/22/2020 17:00,26.0,0.0,0.0
6,2020-01-22,Guangxi,Mainland China,1/22/2020 17:00,2.0,0.0,0.0
7,2020-01-22,Guizhou,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
8,2020-01-22,Hainan,Mainland China,1/22/2020 17:00,4.0,0.0,0.0
9,2020-01-22,Hebei,Mainland China,1/22/2020 17:00,1.0,0.0,0.0


In [4]:
df.isnull().sum()

ObservationDate       0
Province/State     1398
Country/Region        0
Last Update           0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

In [5]:
df[df['Province/State'].isnull()].head(10)

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
35,2020-01-22,NaN,Japan,1/22/2020 17:00,2.0,0.0,0.0
36,2020-01-22,NaN,Thailand,1/22/2020 17:00,2.0,0.0,0.0
37,2020-01-22,NaN,South Korea,1/22/2020 17:00,1.0,0.0,0.0
73,2020-01-23,NaN,Japan,1/23/20 17:00,1.0,0.0,0.0
74,2020-01-23,NaN,Thailand,1/23/20 17:00,3.0,0.0,0.0
75,2020-01-23,NaN,South Korea,1/23/20 17:00,1.0,0.0,0.0
76,2020-01-23,NaN,Singapore,1/23/20 17:00,1.0,0.0,0.0
77,2020-01-23,NaN,Philippines,1/23/20 17:00,0.0,0.0,0.0
78,2020-01-23,NaN,Malaysia,1/23/20 17:00,0.0,0.0,0.0
79,2020-01-23,NaN,Vietnam,1/23/20 17:00,2.0,0.0,0.0


In [6]:
# Current situation
grouped_df = df.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'].max()
grouped_df.style.background_gradient(cmap='Pastel1_r')

In [7]:
# Top 10 Countries with most no. of reported cases
latest_df = df[df['ObservationDate'] == max(df['ObservationDate'])].reset_index()
grouped_df = latest_df.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()

temp = grouped_df[['Country/Region', 'Confirmed']]
temp = temp.sort_values(by='Confirmed', ascending=False)
temp = temp.reset_index(drop=True)
temp.head(10).style.background_gradient(cmap='Pastel1_r')

,Country/Region,Confirmed
0,Mainland China,80652
1,South Korea,7041
2,Italy,5883
3,Iran,5823
4,France,949
5,Germany,799
6,Others,696
7,Spain,500
8,Japan,461
9,US,417


In [8]:
# Countries with all the cases recovered
temp = grouped_df[grouped_df['Confirmed']==grouped_df['Recovered']]
temp = temp[['Country/Region', 'Confirmed', 'Recovered']]
temp = temp.sort_values('Confirmed', ascending=False)
temp = temp.reset_index(drop=True)
temp.style.background_gradient(cmap='Greens')

,Country/Region,Confirmed,Recovered
0,Macau,10,10
1,Cambodia,1,1
2,Nepal,1,1
3,Sri Lanka,1,1


In [9]:
# Most recent stats
world_daily_df = df.groupby('ObservationDate')['Confirmed', 'Deaths', 'Recovered'].sum()
world_daily_df = world_daily_df.reset_index()
world_daily_df = world_daily_df.sort_values('ObservationDate', ascending=False)
world_daily_df.head(1).style.background_gradient(cmap='Pastel1')

,ObservationDate,Confirmed,Deaths,Recovered
45,2020-03-07 00:00:00,105836,3558,58359


In [10]:
world_daily_df.style.background_gradient(cmap='Pastel1')

,ObservationDate,Confirmed,Deaths,Recovered
45,2020-03-07 00:00:00,105836,3558,58359
44,2020-03-06 00:00:00,101800,3460,55866
43,2020-03-05 00:00:00,97886,3348,53797
42,2020-03-04 00:00:00,95124,3254,51171
41,2020-03-03 00:00:00,92844,3160,48229
40,2020-03-02 00:00:00,90309,3085,45602
39,2020-03-01 00:00:00,88371,2996,42716
38,2020-02-29 00:00:00,86013,2941,39782
37,2020-02-28 00:00:00,84124,2872,36711
36,2020-02-27 00:00:00,82756,2814,33277


In [11]:
world_daily_df.isnull().sum()

ObservationDate    0
Confirmed          0
Deaths             0
Recovered          0
dtype: int64

In [12]:
# from https://tykimos.github.io/2017/04/09/RNN_Layer_Talk/
import keras

class LossHistory(keras.callbacks.Callback):
    def init(self):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

Using TensorFlow backend.


In [13]:

import datetime
import os
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint

WINDOW_SIZE = 4

class PandemicRegressor():
    def __init__(self, window_size, batch_size, n_feature, model_name, hidden_state=False, stateful=False, loss_hist=None):
        self.window_size = window_size
        self.batch_size = batch_size
        self.n_feature = n_feature
        save_dir = os.path.join(os.path.join(os.getcwd(), 'model'), 
                                datetime.datetime.now().strftime('%Y-%m-%d;%H.%M.%S'))
        os.mkdir(save_dir)
        self.model_path = os.path.join(save_dir, model_name + '.best.hdf5')
        print(self.model_path)
        self.hidden_state = hidden_state
        self.stateful = stateful
        if stateful:
            self.callbacks = [loss_hist]
        else:
            self.callbacks = [EarlyStopping(monitor='val_mape', patience=100), 
                 ModelCheckpoint(self.model_path, monitor='val_mape', verbose=0, save_best_only=True, mode='min')]
        self.reg = self.build_model(hidden_state, stateful)
        
    def build_model(self, hidden_state, stateful):
        model = Sequential()
        if hidden_state:
            if stateful:
                model.add(LSTM(1024, batch_input_shape=(self.batch_size, self.window_size, self.n_feature), dropout=0.5, stateful=True))
            else:
                model.add(LSTM(1024, input_shape=(self.window_size, self.n_feature), dropout=0.5))
        else:
             model.add(Dense(1024, input_dim=(self.window_size * self.n_feature),activation='relu'))
        
        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.n_feature, activation='linear'))

        model.compile(loss='mean_squared_error', 
                      optimizer='adam',
                      metrics=['mae', 'mape'])
        return model
    
    def reset_states(self):
        # reset hiden states
        self.reg.reset_states()
    
    def fit(self, X_train, y_train, validation_data, epochs=500, verbose=1, shuffle=False):
        hist = self.reg.fit(X_train, 
                            y_train, 
                            epochs=epochs, 
                            batch_size=self.batch_size,
                            validation_data=validation_data,
                            callbacks=self.callbacks,
                            verbose=verbose,
                            shuffle=shuffle)
        return hist
    
    def predict_n_days(self, n, seq_in, scaler):
        _mean = scaler['mean']
        _std = scaler['std']
        
        if self.hidden_state:
            seq_in = list(seq_in)
            seq_out = []
            for i in range(n):
                temp = np.array(seq_in)
                temp = np.reshape(temp, (1, self.window_size, self.n_feature)) # n_sample, seq_len, n_feature
                y_hat = self.reg.predict(temp)
                seq_out.append(y_hat[0])
                y_hat = (y_hat - _mean) / _std
                seq_in.append(y_hat[0])
                seq_in.pop(0)
        else:
            seq_in = list(seq_in)
            seq_out = []
            for i in range(n):
                temp = np.array(seq_in)
                temp = np.reshape(temp, (1, self.window_size * self.n_feature)) # n_sample, seq_len, n_feature
                y_hat = self.reg.predict(temp)
                seq_out.append(y_hat[0])
                y_hat = (y_hat - _mean) / _std
                seq_in.append(y_hat[0])
                seq_in.pop(0)
        seq_out.reverse()
        _df = pd.DataFrame(seq_out)
        _df.columns = ['Confirmed', 'Deaths', 'Recovered']
        return _df
    
    def predict(self, X):
        return self.reg.predict(X)
    
    def evaluate(self, X_test, y_test):
        loss_and_metrics = self.reg.evaluate(X_test, 
                                             y_test, 
                                             batch_size=self.batch_size)
        return loss_and_metrics
    
    def load_best_weights(self):
        self.reg.load_weights(self.model_path)
    
    def save(self):
        self.reg.save(self.model_path)


In [ ]:
res_dir = os.path.join(os.path.join(os.getcwd(), 'result'), 
                                datetime.datetime.now().strftime('%Y-%m-%d;%H.%M.%S'))
os.mkdir(res_dir)

In [14]:
%matplotlib inline
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def show_train_hist(hist):
    plt.plot(hist.history['loss'], 'b-', label='train_loss')
    plt.plot(hist.history['val_loss'], 'r', label='val_loss')
    plt.title("loss history")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Normalize
_mean = np.mean(world_daily_df[['Confirmed', 'Deaths', 'Recovered']].values, axis=0)
_std = np.std(world_daily_df[['Confirmed', 'Deaths', 'Recovered']].values, axis=0)

print(_mean)
print(_std)

[51797.04347826  1535.82608696 16029.95652174]
[34371.13080388  1184.01263772 18420.92444803]


In [15]:
def seq2dataset(sr, window_size):
        seq_data = []
        for i in range(len(sr) - window_size):
            subset = list(sr[i:(i+window_size+1)])
            subset.reverse()
            seq_data.append(subset)
        seq_data.reverse()
        seq_data = np.array(seq_data)
        return seq_data[:,0:window_size,:], seq_data[:,window_size,:]

In [16]:
X, y = seq2dataset(world_daily_df[['Confirmed', 'Deaths', 'Recovered']].values, WINDOW_SIZE)
X

array([[[   555.,     17.,     28.],
        [   653.,     18.,     30.],
        [   941.,     26.,     36.],
        [  1438.,     42.,     39.]],

       [[   653.,     18.,     30.],
        [   941.,     26.,     36.],
        [  1438.,     42.,     39.],
        [  2118.,     56.,     52.]],

       [[   941.,     26.,     36.],
        [  1438.,     42.,     39.],
        [  2118.,     56.,     52.],
        [  2927.,     82.,     61.]],

       [[  1438.,     42.,     39.],
        [  2118.,     56.,     52.],
        [  2927.,     82.,     61.],
        [  5578.,    131.,    107.]],

       [[  2118.,     56.,     52.],
        [  2927.,     82.,     61.],
        [  5578.,    131.,    107.],
        [  6165.,    133.,    126.]],

       [[  2927.,     82.,     61.],
        [  5578.,    131.,    107.],
        [  6165.,    133.,    126.],
        [  8235.,    171.,    143.]],

       [[  5578.,    131.,    107.],
        [  6165.,    133.,    126.],
        [  8235.,    171.,

In [17]:
y

array([[  2118.,     56.,     52.],
       [  2927.,     82.,     61.],
       [  5578.,    131.,    107.],
       [  6165.,    133.,    126.],
       [  8235.,    171.,    143.],
       [  9925.,    213.,    222.],
       [ 12038.,    259.,    284.],
       [ 16787.,    362.,    472.],
       [ 19881.,    426.,    623.],
       [ 23892.,    492.,    852.],
       [ 27636.,    564.,   1124.],
       [ 30818.,    634.,   1487.],
       [ 34392.,    719.,   2011.],
       [ 37121.,    806.,   2616.],
       [ 40151.,    906.,   3244.],
       [ 42763.,   1013.,   3946.],
       [ 44803.,   1113.,   4683.],
       [ 45222.,   1118.,   5150.],
       [ 60370.,   1371.,   6295.],
       [ 66887.,   1523.,   8058.],
       [ 69032.,   1666.,   9395.],
       [ 71226.,   1770.,  10865.],
       [ 73260.,   1868.,  12583.],
       [ 75138.,   2007.,  14352.],
       [ 75641.,   2122.,  16121.],
       [ 76199.,   2247.,  18177.],
       [ 76843.,   2251.,  18890.],
       [ 78599.,   2458.,  2

In [18]:
X = (X - _mean) / _std

In [19]:
X

array([[[-1.49084544, -1.28277861, -0.86868368],
        [-1.48799421, -1.28193403, -0.86857511],
        [-1.47961508, -1.27517734, -0.8682494 ],
        [-1.46515527, -1.26166397, -0.86808654]],

       [[-1.48799421, -1.28193403, -0.86857511],
        [-1.47961508, -1.27517734, -0.8682494 ],
        [-1.46515527, -1.26166397, -0.86808654],
        [-1.44537123, -1.24983977, -0.86738082]],

       [[-1.47961508, -1.27517734, -0.8682494 ],
        [-1.46515527, -1.26166397, -0.86808654],
        [-1.44537123, -1.24983977, -0.86738082],
        [-1.42183403, -1.22788055, -0.86689224]],

       [[-1.46515527, -1.26166397, -0.86808654],
        [-1.44537123, -1.24983977, -0.86738082],
        [-1.42183403, -1.22788055, -0.86689224],
        [-1.34470535, -1.18649586, -0.86439508]],

       [[-1.44537123, -1.24983977, -0.86738082],
        [-1.42183403, -1.22788055, -0.86689224],
        [-1.34470535, -1.18649586, -0.86439508],
        [-1.32762706, -1.18480668, -0.86336365]],

       [[-

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=False)

In [25]:
"""
Predict the global spread of coronavirus with LSTM modle
"""
pdm_reg = PandemicRegressor(window_size=WINDOW_SIZE, hidden_state=True, batch_size=8, n_feature=3, model_name='world_lstm_pdm_reg', stateful=False)

hist = pdm_reg.fit(X_train=X_train, 
                   y_train=y_train, 
                   validation_data=(X_test, y_test),
                   verbose=1,
                   epochs=3000)
show_train_hist(hist)

pdm_reg.load_best_weights()
loss_and_metrics = pdm_reg.evaluate(X_test=X_test, y_test=y_test)

C:\Users\codez\PycharmProjects\CoronavirusDisease2019\model\2020-03-08;21.59.34\world_lstm_pdm_reg.best.hdf5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 35 samples, validate on 7 samples
Epoch 1/3000


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(8, 3), b.shape=(3, 4096), m=8, n=4096, k=3
	 [[{{node lstm_1/while/MatMul}}]]
	 [[metrics/mape/Identity/_157]]
  (1) Internal: Blas GEMM launch failed : a.shape=(8, 3), b.shape=(3, 4096), m=8, n=4096, k=3
	 [[{{node lstm_1/while/MatMul}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
print(f'MAE: {loss_and_metrics[1]}, MAPE: {loss_and_metrics[2]}')
print('===================================================== Prediction of X_test =====================================================')
y_hat = pdm_reg.predict(X_test)
y_dict = {'Confirmed': [x[0] for x in y_hat],
          'Deaths':[x[1] for x in y_hat],
          'Recovered':[x[2] for x in y_hat]}
y_df = pd.DataFrame(y_dict)
y_df

In [ ]:
from datetime import timedelta

date_index = world_daily_df['ObservationDate'] + timedelta(days=n)
date_index[:n]

pd.set_option('display.float_format', lambda x: '%.3f' % x)
n = 7
print(f'\n=============================================== World Prediction of {n} days ======
prediction = pdm_reg.predict_n_days(n, X_test[-1], {'mean': _mean, 'std': _std})
prediction = prediction.set_index(date_index[:n])
prediction

In [ ]:
prediction.to_csv(os.path.join(res_dir, f'world_{n}days_prediction.csv'))

In [ ]:
print(df['Country/Region'].unique())
nation_name = 'South Korea'

In [ ]:
korea_df = df[df['Country/Region'] == nation_name].groupby('ObservationDate')['Confirmed', 'Deaths', 'Recovered'].sum()
korea_df = korea_df.reset_index()
korea_df = korea_df.sort_values('ObservationDate', ascending=False)
korea_df.style.background_gradient(cmap='Pastel1')

In [ ]:
# for nation_name in df['Country/Region'].unique():
print(f'[{nation_name} spread of coronavirus19]')
_mean = np.mean(korea_df[['Confirmed', 'Deaths', 'Recovered']].values, axis=0)
_std = np.std(korea_df[['Confirmed', 'Deaths', 'Recovered']].values, axis=0)
    
X, y = seq2dataset(korea_df[['Confirmed', 'Deaths', 'Recovered']].values, WINDOW_SIZE)
X = (X - _mean) / _std
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=False)

In [ ]:
"""
Predict the korea spread of coronavirus with Dense modle
"""
k_pdm_reg = PandemicRegressor(window_size=WINDOW_SIZE, hidden_state=True, batch_size=8, n_feature=3, model_name='korea_lstm_pdm_reg', stateful=False)

hist = k_pdm_reg.fit(X_train=X_train,
                   y_train=y_train, 
                   validation_data=(X_test, y_test),
                   verbose=1,
                   epochs=3000)
show_train_hist(hist)
k_pdm_reg.load_best_weights()
loss_and_metrics = k_pdm_reg.evaluate(X_test=X_test, y_test=y_test)

In [ ]:
print(f'MAE: {loss_and_metrics[1]}, MAPE: {loss_and_metrics[2]}')
print('===================================================== Prediction of X_test =====================================================')
y_hat = k_pdm_reg.predict(X_test)
y_dict = {'Confirmed': [x[0] for x in y_hat],
          'Deaths':[x[1] for x in y_hat],
          'Recovered':[x[2] for x in y_hat]}
y_df = pd.DataFrame(y_dict)
y_df

In [ ]:
from datetime import timedelta

date_index = korea_df['ObservationDate'] + timedelta(days=n)
date_index[:n]

pd.set_option('display.float_format', lambda x: '%.3f' % x)
n = 7
print(f'\n============================================{nation_name} Prediction of {n} days =====================================================')
prediction = k_pdm_reg.predict_n_days(n, X_test[-1], {'mean': _mean, 'std': _std})
prediction = prediction.set_index(date_index[:n])
prediction

In [ ]:
prediction.to_csv(os.path.join(res_dir, f'korea_{n}days_prediction.csv'), index=True)